In [1]:
import numpy as np
import pyrealsense2 as rs
import cv2
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
# Initialize the RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable depth and color streams with desired settings
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
path = '/home/rpmdt05/Code/break-it/nirschal/mediapipe/visualization_demo/demo_rgb_video.avi'

writer = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'MJPG'), 30, (640, 480))

# color_image_data = []
# depth_image_data = []

color_image_data = np.empty((2000, 480, 640, 4), dtype=np.uint8)
point_cloud_data = []


# Start the pipeline with the configured settings
pipeline.start(config)

# Create align object
align_to = rs.stream.color
align = rs.align(align_to)
# try:
iter = 0
while True:
    # Wait for frames
    frames = pipeline.wait_for_frames()
    
    # Align the frames
    aligned_frames = align.process(frames)

    color_frame = aligned_frames.get_color_frame()
    depth_frame = aligned_frames.get_depth_frame()

    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    color_image_data[iter,:,:,0:3] = color_image
    color_image_data[iter,:,:,3] = depth_image
    iter += 1
    writer.write(color_image)

    cv2.imshow('Color Stream', color_image)
    cv2.imshow('Depth Stream', depth_image)
    # Break the loop if 'q' is pressed

    pc = rs.pointcloud()
    pc.map_to(color_frame)
    points = pc.calculate(depth_frame)
    point_cloud = np.asanyarray(points.get_vertices())
    point_cloud_data.append(point_cloud)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        writer.release()
        pipeline.stop()

        # Close all OpenCV windows
        cv2.destroyAllWindows()
        break


In [4]:
save_npy = np.save('visualization_demo/color_image_data_with_depth_4_channels.npy', color_image_data)
save_point_cloud = np.save('visualization_demo/point_cloud_data.npy', point_cloud_data)

In [6]:
input_path = '/home/rpmdt05/Code/break-it/nirschal/mediapipe/action classification dataset/pulling/videos/1.avi'
output_path_mp = '/home/rpmdt05/Code/break-it/nirschal/mediapipe/visualization_demo/demo_mediapipe.avi'
output_path_landmark = '/home/rpmdt05/Code/break-it/nirschal/mediapipe/visualization_demo/demo_landmark.avi'

righthand_landmarks = []
lefthand_landmarks = []
writer = cv2.VideoWriter(output_path_mp, cv2.VideoWriter_fourcc(*'MJPG'), 30, (640, 480))
video = cv2.VideoCapture(input_path)
while True:
    ret, frame = video.read()
    if not ret:
        cv2.destroyAllWindows()
        writer.release()
        break

    '''Media Pipe will sometime detect both hands as right hand.'''

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2) as hands:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        print("Im am here :", len(results.multi_hand_landmarks))
        if len(results.multi_hand_landmarks)==2 and (results.multi_handedness[0].classification[0].label != results.multi_handedness[1].classification[0].label):
            for num,hand in enumerate(results.multi_hand_landmarks):
                # print("num: ", num, "hand: ", hand)
                temp = []
                handType = results.multi_handedness[num].classification[0].label
                
                # if len(results.multi_handedness[num].classification) == 2:
                if handType == 'Right':
                    drawing_spec = mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                    for i in range(len(hand.landmark)):
                        temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                    righthand_landmarks.append(np.array(temp))

                elif handType == 'Left':
                    drawing_spec = mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2)
                    for i in range(len(hand.landmark)):
                        temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                    lefthand_landmarks.append(temp)

                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, drawing_spec, drawing_spec)

        elif len(results.multi_hand_landmarks)==2 and (results.multi_handedness[0].classification[0].label == results.multi_handedness[1].classification[0].label):
            '''Both hands were detected as right or left here need to fix this.'''
            for num,hand in enumerate(results.multi_hand_landmarks):
                # print("num: ", num, "hand: ", hand)
                temp = []
                handType = results.multi_handedness[num].classification[0].label
                
                # if len(results.multi_handedness[num].classification) == 2:
                if handType == 'Right':
                    drawing_spec = mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                    for i in range(len(hand.landmark)):
                        temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                    righthand_landmarks.append(np.array(temp))
                    temp2 = np.zeros_like(temp)
                    lefthand_landmarks.append(temp2)

                elif handType == 'Left':
                    drawing_spec = mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2)
                    for i in range(len(hand.landmark)):
                        temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                    lefthand_landmarks.append(temp)
                    temp2 = np.zeros_like(temp)
                    righthand_landmarks.append(temp2)

                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, drawing_spec, drawing_spec)

            pass
        elif len(results.multi_hand_landmarks)==1:
            hand = results.multi_hand_landmarks[0]
            # print("Nirschal : ", hand)
            handType = results.multi_handedness[0].classification[0].label
            # print("handType : ", handType)
            temp  = []
            if handType == 'Right':
                drawing_spec = mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                for i in range(len(hand.landmark)):
                    temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                righthand_landmarks.append(np.array(temp))

                temp2 = np.zeros_like(temp)
                lefthand_landmarks.append(temp2)

            if handType == 'Left':
                drawing_spec = mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2)
                for i in range(len(hand.landmark)):
                    temp.append(np.array([hand.landmark[i].x, hand.landmark[i].y, hand.landmark[i].z]))
                lefthand_landmarks.append(np.array(temp))

                temp2 = np.zeros_like(temp)
                righthand_landmarks.append(temp2)
            
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, drawing_spec, drawing_spec)

        elif results.multi_hand_landmarks==None:
            righthand_landmarks.append(np.zeros((21, 3)))
            lefthand_landmarks.append(np.zeros((21, 3)))


    cv2.namedWindow("MediaPipe Hands", cv2.WINDOW_NORMAL) 
    # Using resizeWindow() 
    cv2.resizeWindow("MediaPipe Hands", 640, 480) 
    cv2.imshow('MediaPipe Hands', image)
    writer.write(image)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        writer.release()

        break

I0000 00:00:1712787008.352738 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787008.374522 2551740 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
I0000 00:00:1712787008.453773 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787008.539019 2551767 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2


Im am here : 2
Im am here : 2


I0000 00:00:1712787008.665058 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787008.755512 2551795 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
I0000 00:00:1712787008.886325 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5


Im am here : 2


I0000 00:00:1712787008.989479 2551823 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
I0000 00:00:1712787009.116767 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787009.204644 2551852 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2


...

I0000 00:00:1712787025.781565 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787025.889404 2555032 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2


Im am here : 2


I0000 00:00:1712787026.016771 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787026.105886 2555060 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2


Im am here : 2
Im am here : 2


I0000 00:00:1712787026.243019 2530217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712787026.355747 2555088 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2


In [ ]:
righthand_landmarks = np.array(righthand_landmarks)
lefthand_landmarks = np.array(lefthand_landmarks)
combined_landmarks = np.concatenate((righthand_landmarks, lefthand_landmarks), axis=1)
np.save(output_path_landmark, combined_landmarks)
print("combined_landmarks shape: ", combined_landmarks.shape, "righthand_landmarks shape: ", righthand_landmarks.shape, "lefthand_landmarks shape: ", lefthand_landmarks.shape)

In [ ]:
thumh_of_righthand = righthand_landmarks[:, 4, :]

for i,hand in enumerate(righthand_landmarks):
    print(tuple([hand[4, 0], hand[4, 1], hand[4, 2]]))

In [ ]:
thumh_of_left = lefthand_landmarks[:, 4, :]

for i,hand in enumerate(lefthand_landmarks):
    print(tuple([hand[4, 0], hand[4, 1], hand[4, 2]]))

# Convert the depth images to Pointcloud

In [ ]:
def depth2pc(depth_image, intrinsics):
    # Get the shape of the depth image
    h, w = depth_image.shape

    # Generate the pixel grid
    x = np.linspace(0, w-1, w)
    y = np.linspace(0, h-1, h)
    x, y = np.meshgrid(x, y)

    # Apply the depth image to the grid
    X = (x - intrinsics.ppx) / intrinsics.fx
    Y = (y - intrinsics.ppy) / intrinsics.fy
    Z = depth_image / 1000

    # Reshape the data
    X = X.reshape(-1)
    Y = Y.reshape(-1)
    Z = Z.reshape(-1)

    # Stack the data
    pc = np.vstack((X, Y, Z)).T

    return pc

In [ ]:
#load the images from a npy file
images = np.load('color_image_data_2.npy')

color_images = images[:,:,:,:3]
depth_images = images[:,:,:,3]


In [ ]:
print(depth_images[0])

In [ ]:
pointclouds = [depth2pc(depth_image, rs.intrinsics) for depth_image in depth_images]

In [ ]:
# run mediapipe on all the color images and visualise the landmarks on the color images using cv2.imshow, and also store the processed images the same way as the color images "a list of images"

# run mediapipe on all the color images and visualise the landmarks on the color images using cv2.imshow, and also store the processed images the same way as the color images "a list of images"
import mediapipe as mp
# import cv2
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

processed_images = []

for image in color_images:
    processed_image = image.copy()
    with mp_hands.Hands(static_image_mode=True,max_num_hands=2,min_detection_confidence=0.5) as hands:
        # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # print(results.multi_hand_landmarks)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    processed_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    processed_images.append(processed_image)          
    # cv2.imshow('MediaPipe Hands', processed_image)
    # cv2.imshow('Original Image', image)
    # cv2.waitKey(5)
cv2.destroyAllWindows()